In [ ]:
!pip install opencv-python numpy matplotlib scipy

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2, fftshift, ifftshift
import os
import random

input_dir = 'exercise'
output_dir = 'output_fft_combined'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def random_rgb_order(img):
    channels = list(cv2.split(img))
    random.shuffle(channels)
    return cv2.merge(channels)

def fast_fourier(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    f = fft2(gray)
    fshift = fftshift(f)
    magnitude = 20 * np.log(np.abs(fshift) + 1)
    return np.uint8(cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX))

def butterworth_filter(img, d0, n=2, highpass=False):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rows, cols = gray.shape
    u = np.arange(rows)
    v = np.arange(cols)
    u[u > rows // 2] -= rows
    v[v > cols // 2] -= cols
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2 + V**2)
    if highpass:
        H = 1 / (1 + (d0 / (D + 1e-5))**(2 * n))
    else:
        H = 1 / (1 + (D / d0)**(2 * n))
    F = fftshift(fft2(gray))
    G = H * F
    g = np.abs(ifft2(ifftshift(G)))
    return np.uint8(cv2.normalize(g, None, 0, 255, cv2.NORM_MINMAX))

def butterworth_lowpass(img):
    return butterworth_filter(img, d0=50, n=2, highpass=False)

def butterworth_highpass(img):
    return butterworth_filter(img, d0=50, n=2, highpass=True)

def min_filter(img):
    return cv2.erode(img, np.ones((3, 3), np.uint8))

def max_filter(img):
    return cv2.dilate(img, np.ones((3, 3), np.uint8))

In [ ]:
transformations = {
    'F': ('Fast Fourier Transform', fast_fourier),
    'L': ('Butterworth Lowpass + Min Filter', lambda img: min_filter(butterworth_lowpass(img))),
    'H': ('Butterworth Highpass + Max Filter', lambda img: max_filter(butterworth_highpass(img)))
}

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
        path = os.path.join(input_dir, filename)
        img = cv2.imread(path)

        shuffled = random_rgb_order(img)

        code = random.choice(list(transformations.keys()))
        name, transform_fn = transformations[code]
        print(f"Đang xử lý {filename} với: {name}")

        result = transform_fn(shuffled)
        out_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_{code}.png")
        cv2.imwrite(out_path, result)

        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(shuffled, cv2.COLOR_BGR2RGB))
        plt.title("RGB Shuffled")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(result, cmap='gray')
        plt.title(name)
        plt.axis("off")
        plt.show()

print(f"✅ Hoàn tất. Ảnh đã lưu tại: {output_dir}")